In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import time

In [3]:
from gpt_translate.articles.JsonArticleManager import JsonArticleManager
from gpt_translate.crawl.util import extract_info, scroll_one_step
import pandas as pd

In [22]:
ARTICLE_JSON_PATH = "/home/bookworm/code/gpt_translate/data/articles_embedding.json"
article_manager = JsonArticleManager(ARTICLE_JSON_PATH)

In [25]:
article_manager.articles_df.tail()

,title,date,url,is_original,text,text_len,translation,english_tags,chinese_tags,english_main_topic,chinese_main_topic,media_ct,chinese_text_len,pre_embed_tokens,embedding,id,english_title
2520,国内韭菜图鉴,2100-01-01,https://m.163.com/news/article/JLDFO8CU0553JMK...,NaN,\n关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》继续那天第一部分的基本...,NaN,"Follow the official account, ""人间罗盘"" (Compass o...","{'Emotions': 7, 'Professional Investors': 8, '...","{'情绪': 7, '专业投资者': 8, '散户投资者': 6, '麻将': 5, '新闻...",Investment Strategies,投资策略,NaN,NaN,Investment_Strategies Emotions Professional_In...,"[-0.0243285634, -0.0187849998, 0.0234366544, -...",2520,Domestic Leek Plant Guide
2521,哪怕你不欣赏他，你也很清楚，这个社会欣赏他,2100-01-01,https://m.163.com/news/article/JLENI1P20553JMK...,NaN,\n关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》昨天我聊无人机专家的时...,NaN,"Follow the official account, ""人间罗盘"" (Compass o...","{'Achievement': 8, 'Goal Setting': 7, 'Practic...","{'成就': 8, '目标设定': 7, '实事求是': 9, '成功': 6, '自我提升...",Achieving Goals and Being Practical,实现目标和实事求是,NaN,NaN,Achieving_Goals_and_Being_Practical Achievemen...,"[-0.017512504, 0.0061864066, 0.0306294933, -0....",2521,"Even if you don't appreciate him, you are very..."
2522,已访问浙大的段永平，拟收购格陵兰的特朗普,2100-01-01,https://m.163.com/news/article/JLF6GF7B0553JMK...,NaN,\n关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》有人看了上午那个关于送...,NaN,"Pay attention to the public account, ""Human Co...","{'Gift-giving': 7, 'Duan Yongping': 8, 'Territ...","{'礼物': 7, '段永平': 8, '领土': 6, '特朗普': 9, '土地': 7...",Business and Commercial Civilization,商业和商业文明,NaN,NaN,Business_and_Commercial_Civilization Gift-givi...,"[-0.0183467958, -0.0240966622, 0.0210272893, -...",2522,"Duan Yongping, who has visited Zhejiang Univer..."
2523,我没有在说反话，洛杉矶大火还真体现了美国的优势,2100-01-01,https://m.163.com/news/article/JLHJ13520553JMK...,NaN,\n关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》我那天在做2025全年...,NaN,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》。\n\n我那天在做202...,"{'Economic Analysis': 8, 'Labor Market': 7, 'G...","{'经济分析': 8, '劳动力市场': 7, '全球化': 6, '贸易': 6, '需求...",Inflation,通胀,NaN,NaN,Inflation Economic_Analysis Labor_Market Globa...,"[-0.0193724148, -0.0136978766, 0.005116784, -0...",2523,"I'm not being sarcastic, the Los Angeles wildf..."
2524,无儿无女的86岁北大老太太晚年幸福么？,2100-01-01,https://m.163.com/news/article/JLHE6B970553JMK...,NaN,\n关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》那天有读者问我这么一个...,NaN,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》。\n\n那天有读者问我这...,"{'Life Reflection': 8, 'Work-Life Balance': 7,...","{'人生反思': 8, '工作与生活平衡': 7, '家庭': 6, '婚姻': 5, '职...",Time Management,时间管理,NaN,NaN,Time_Management Life_Reflection Work-Life_Bala...,"[8.59042e-05, -0.0008544576000000001, 0.010164...",2524,Is the 86-year-old Peking University lady with...


In [6]:
#num_articles = ...  # Set the desired number of articles
url = "https://c.m.163.com/news/sub/T1658526449605.html"

# Set up headless browser
options = Options()
options.add_argument('--headless')
#options.headless = True
driver = webdriver.Chrome(options=options)

# Load webpage and extract data
driver.get(url)

In [6]:
#scroll_one_step(driver)

In [7]:
# # Get current page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for the page to load
    time.sleep(2)

    # Get new page height after scrolling
    new_height = driver.execute_script("return document.body.scrollHeight")

    # If the page height has not changed, we've reached the end of the page
    if new_height == last_height:
        break
    last_height = new_height

In [8]:
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Find all elements with the specified class
elements = soup.find_all("li", class_="single-picture-news js-click-news")
len(elements)

999

In [9]:
articles_info = pd.DataFrame(extract_info(elements)).drop_duplicates(['title'])
articles_info.shape

(964, 4)

In [10]:
urls = articles_info['link'].apply(lambda url: "https://" + url.replace("//","")).head().tolist()
len(urls)

5

In [11]:
len(set(articles_info['title']).difference(set(article_manager.articles_df['title'])))

7

In [12]:
existing_titles = set(article_manager.articles_df['title'])

In [13]:
new_articles_df = articles_info[~articles_info['title'].isin(existing_titles)]

In [14]:
url_to_dictionary = {}

In [15]:
if new_articles_df.shape[0]:
    urls = new_articles_df['link'].apply(lambda url: "https://" + url.replace("//","")).tolist()
    for url in urls:
        try:
            driver.get(url)
            
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Find all elements with the specified class

            content_div = soup.find("div", class_="article-body")
            element_text = content_div.get_text()
            reformmated_text = element_text.replace("。", "\n")
            
            url_to_dictionary[url] = reformmated_text
        except Exception as e:
            print(f"error processing {url}")
            print(e)

In [16]:
new_articles_df.head()

,title,link,image_url,date
0,真正让你抑郁的不是心理疾病，是穷,//c.m.163.com/news/a/JLQ8GGF80553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0113/244013f3j0...,12小时前
1,真正让你痛苦的不只是失业，更是失去了身份认同,//c.m.163.com/news/a/JLP4SLC00553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0113/fa65a105j0...,22小时前
2,没有利益上的关联，人与人之间就无所谓关系,//c.m.163.com/news/a/JLNQN3BU0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0112/b466d22cj0...,1天前
3,为什么美股涨的这么稳定，美国的穷人却没几个翻身？,//c.m.163.com/news/a/JLMJ9S0V0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0112/4c950309j0...,1天前
4,像你这么勤劳的员工，搁我手里也是不会被提拔的,//c.m.163.com/news/a/JLL8PTMK0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0111/d90979c3j0...,2025-01-11


In [17]:
len(url_to_dictionary)

7

In [18]:
new_articles_df['url'] = new_articles_df['link'].apply(lambda url: "https://" + url.replace("//",""))

/tmp/ipykernel_271743/3054605342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_articles_df['url'] = new_articles_df['link'].apply(lambda url: "https://" + url.replace("//",""))


In [19]:
new_articles_df['text'] = new_articles_df['link'].apply(lambda url: "https://" + url.replace("//","")).apply(lambda url: url_to_dictionary.get(url, "")).\
    apply(lambda text:
         text.replace("文章来自微信公众号：记忆承载\n欢迎前往关注阅读全文\n", "").\
          replace("\n文章来自微信公众号：记忆承载\n欢迎前往关注阅读全文\n\n\n\n\n打开网易新闻 查看更多图片", "")
         )

/tmp/ipykernel_271743/3349361820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_articles_df['text'] = new_articles_df['link'].apply(lambda url: "https://" + url.replace("//","")).apply(lambda url: url_to_dictionary.get(url, "")).\


In [21]:
new_articles_df

,title,link,image_url,date,url,text
0,真正让你抑郁的不是心理疾病，是穷,//c.m.163.com/news/a/JLQ8GGF80553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0113/244013f3j0...,12小时前,https://c.m.163.com/news/a/JLQ8GGF80553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》经常有读者留言向我咨询心理...
1,真正让你痛苦的不只是失业，更是失去了身份认同,//c.m.163.com/news/a/JLP4SLC00553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0113/fa65a105j0...,22小时前,https://c.m.163.com/news/a/JLP4SLC00553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》有读者那天问我，第四部分我...
2,没有利益上的关联，人与人之间就无所谓关系,//c.m.163.com/news/a/JLNQN3BU0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0112/b466d22cj0...,1天前,https://c.m.163.com/news/a/JLNQN3BU0553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》那天有人问我这么一个问题\...
3,为什么美股涨的这么稳定，美国的穷人却没几个翻身？,//c.m.163.com/news/a/JLMJ9S0V0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0112/4c950309j0...,1天前,https://c.m.163.com/news/a/JLMJ9S0V0553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》回答一个满级读者的问题，他...
4,像你这么勤劳的员工，搁我手里也是不会被提拔的,//c.m.163.com/news/a/JLL8PTMK0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0111/d90979c3j0...,2025-01-11,https://c.m.163.com/news/a/JLL8PTMK0553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》前天有读者表示不同意见，他...
5,这是你的弱项？不，这分明是老天留给你的福报啊,//c.m.163.com/news/a/JLJVN96G0553JMK2.html?fro...,http://dingyue.ws.126.net/2025/0111/7155052ej0...,2025-01-11,https://c.m.163.com/news/a/JLJVN96G0553JMK2.ht...,关注公众号，人间罗盘，方能阅读正文《2025，不要一年到头又白忙》那天有个读者看了第四个话题...
195,你看着你自己，也看着众生，忽然莞尔一笑,//c.m.163.com/news/a/JDG7FPO00553JMK2.html?fro...,https://nimg.ws.126.net/?url=http%3A%2F%2Fding...,2024-10-02,https://c.m.163.com/news/a/JDG7FPO00553JMK2.ht...,关注公众号，西风的罗盘，方能阅读正文《什么是天机？什么又是泄露天机？》其实那天聊天机时，我觉...


In [20]:
new_articles_df.to_csv("./new_articles_df_.csv", index=False)

In [41]:
driver.quit()

#### article completion

In [65]:
records = new_articles_df.to_dict("records")
records[0]

{'title': '你要是知道自己想干嘛，就不会每天被核污染吓得要死',
 'link': '//c.m.163.com/news/a/ID297MHD0553JMK2.html?from=subscribe',
 'image_url': 'http://dingyue.ws.126.net/2023/0826/37b338b3j00rzz3g7009bd000n300cxp.jpg',
 'date': '3小时前',
 'text': '文章来自微信公众号：记忆承载3，请关注阅读全文\n昨天看了我们不惹事，但也不怕事，有感于日本排放核污染\n有读者跟我讲，他还是很恐惧\n不理解我这种一代人只能做一代人的事的乐观精神是哪儿来的\n似乎每每遇到重大的危机，我身上都没有什么负面情绪，而是迅速能够找到该做的，或者说，是我想做的\n为什么？这个问题有人替我回答过，很多年前的一本神剧，《寻秦记》\n有印象的人，应该记得项少龙就是在大家面前讲了这么一段有关人生意义的寓言，从而赢得了琴清姑娘的芳心\n“有个旅客在沙漠里遇上了一群饿狼，狼群对他展开了攻击\n旅客非常害怕，但为了活下去只得夺路狂奔\n就在饿狼快追上他时，他跑到了一口深不见底的井旁，为了保命不顾一切跳了进去\n在下坠的时候，他胡乱地用手在空中挥舞，祈祷能抓到救命的稻草，没想到，竟然真的被他抓住了一株长在井壁上的小树\n他抓住树干悬在半空，低头向下看去，原来这口井早已干枯，而且井底竟爬满了毒蛇\n上有饿狼，下有毒蛇，那人虽陷身在进退两难的绝境，但暂时却是安全的\n就在旅人长舒口气，庆幸自己保住性命的时候，他听到了奇怪的声音\n循声看去，发现竟然有几只老鼠露出大门牙在不断啃着树枝，旅人顿时感到无比的绝望，就在这生死一瞬的时刻，他看到了眼前的树叶上竟然有一滴蜜糖，那一瞬间，他忘记了一切，全心全意去品尝那滴蜜糖\n那滴蜜糖就是生命的意义！"我昨天告诉你的就这点事儿\n我问你的只有一句话，此时此刻，你想做点什么？琴清这样的美女，才女，喜欢项少龙没什么稀奇，很多人都喜欢他，因为在他身上，我们能够看到生命力\n我们都喜欢那个有生命力的人，而不是暮气沉沉\n王阳明说得对，世界到底是丛林，还是迪斯尼乐园，根本就不重要\n重要的是，你怎么看，你怎么想，你到底想玩点啥\n无论世界是丛林，还是迪斯尼乐园，都是一个游戏\n残酷的游戏是游戏

In [54]:
import openai
import os
import json

openai.api_key = os.environ["OPENAI_API_KEY"]
completion_config = {
    'model': "gpt-4",
    'temperature': 0
}

In [68]:
for record in records[1:]:
    try:
        if record['text']:
            print(f"processing record: {record['title']}")
            record['text'] = record['text'].replace("\n\n","\n")
            new_record = article_manager.complete_article(record, completion_config)
            article_manager.add_complete_article(new_record)
        else:
            continue
    except Exception as e:
        print(f"error processing record {record['title']}")
        print(e)

processing record: 男子交了10万彩礼，举办订婚宴后，反被告强奸


Sending chatcomplete API requests:   0%|          | 0/1 [00:00<?, ?it/s]

Sending chatcomplete API requests: 100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


split test into trunks
there are 4 text_trunks


Sending chatcomplete API requests: 100%|██████████| 1/1 [00:15<00:00, 15.22s/it]
/opt/shichenh/gpt_translate/src/gpt_translate/articles/JsonArticleManager.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.articles_df = self.articles_df.append(series, ignore_index=True)


...

Sending chatcomplete API requests: 100%|██████████| 1/1 [00:15<00:00, 15.15s/it]
/opt/shichenh/gpt_translate/src/gpt_translate/articles/JsonArticleManager.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.articles_df = self.articles_df.append(series, ignore_index=True)


processing record: 从这个月薪八万炫富停职的90后来看工资保密制度


Sending chatcomplete API requests: 100%|██████████| 1/1 [00:02<00:00,  2.98s/it]


split test into trunks
there are 4 text_trunks


Sending chatcomplete API requests: 100%|██████████| 1/1 [00:12<00:00, 12.76s/it]
/opt/shichenh/gpt_translate/src/gpt_translate/articles/JsonArticleManager.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.articles_df = self.articles_df.append(series, ignore_index=True)


In [69]:
article_manager.articles_df.shape

(1580, 17)

In [72]:
from gpt_translate.crawl.util import normalize_date
article_manager.articles_df['date'] = article_manager.articles_df['date'].apply(normalize_date)

In [75]:
article_manager.articles_df.to_json(article_manager.article_json_path, orient='records')